# Importing Libraries

In [ ]:
# Imports
import os
import pandas as pd
import numpy as np
import torch
import pickle
!python3 -m pip install torchmetrics
from nltk.translate.bleu_score import sentence_bleu
from torchmetrics.text import WordErrorRate, BERTScore
from torchmetrics.text.rouge import ROUGEScore
# from nltk.translate.meteor_score import meteor_score

# Toggle for loading general roman urdu model or saving general roman urdu model after training
# If we are working on english, then also we should follow save general strategy
loadGeneral, saveGeneral = 1, 0

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


# Reading data from Google Drive (Back up option. GitHub preffered)

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')
# os.chdir("/content/drive/My Drive/Colab Notebooks")
# path = "/content/drive/My Drive/Kaavish-Mai-Transformer-Based-Urdu-Chatbot/Datasets/"

# Cloning GitHub Repository

In [ ]:
!git clone https://github.com/ShafaqFatimaMughal/Kaavish-Mai-Transformer-Based-Urdu-Chatbot.git
path = "./Kaavish-Mai-Transformer-Based-Urdu-Chatbot/Datasets"

fatal: destination path 'Kaavish-Mai-Transformer-Based-Urdu-Chatbot' already exists and is not an empty directory.


# Installings and Loading DialoGPT pretrained model

In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer

In [ ]:
if loadGeneral:
  model = pickle.load(open('mai_general_model.pkl', 'rb'))
else:
  model = AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-small")

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")

In [ ]:
"""
Fine-tuning the library models for language modeling on a text file (GPT, GPT-2, BERT, RoBERTa).
GPT and GPT-2 are fine-tuned using a causal language modeling (CLM) loss while BERT and RoBERTa are fine-tuned
using a masked language modeling (MLM) loss.
"""
import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import pandas as pd
import numpy as np
import torch

from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

from pathlib import Path

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

# Configs
logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

In [ ]:
# Args to allow for easy convertion of python script to notebook
class Args():
    def __init__(self):
        self.output_dir = 'output-small'
        self.model_type = 'gpt2'
        self.model_name_or_path = 'microsoft/DialoGPT-small'
        self.config_name = 'microsoft/DialoGPT-small'
        self.tokenizer_name = 'microsoft/DialoGPT-small'
        self.cache_dir = 'cached'
        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 4
        self.per_gpu_eval_batch_size = 4
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 25
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 3500
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'

args = Args()

# Reading Files and Creating Merged Dataset

In [ ]:
Missed = 0

if saveGeneral:
    df1 = pd.read_csv("./DataSets/General_Conversation_compiled.csv")
else:
    df1 = pd.read_csv('./DataSets/self_curated_roman_urdu.csv')

# Checking instance lengths to avoid memory errors during training
count = 0
for index, row in df1.iterrows():
    print(row)
    if len("".join([row[i] for i in df1.columns])) > 1024:
        count += 1

print(count)

# df1.info()

response     jab aurat ki andaam nihani/vagina se kuch dino...
context                                      mahvari kya hai ?
context/0    mein samjhti hon ke aap ko aisa kyun lagta hai...
context/1            kyunkay mujhe mahvari ki samajh nahi aati
context/2                         aap ko aisa kyun lagta hai ?
context/3                     mujhe baychaini mehsoos hoti hai
context/4         aaj mein aap ki madad kaisay kar sakti hon ?
context/5                                               Hi Mai
Name: 0, dtype: object
response     mujhe mahvari ho gayi hai aur mein ne abhi tak...
context      jab aurat ki andaam nihani/vagina se kuch dino...
context/0                                    mahvari kya hai ?
context/1    mein samjhti hon ke aap ko aisa kyun lagta hai...
context/2            kyunkay mujhe mahvari ki samajh nahi aati
context/3                         aap ko aisa kyun lagta hai ?
context/4                     mujhe baychaini mehsoos hoti hai
context/5         aaj mein aap k

response                                 aap ka naam kya hai ?
context                                             jee zarur!
context/0          kya mein aap se aik sawal pooch sakti hon ?
context/1      hi. aaj mein aap ki madad kaisay karsakti hon .
context/2                                                hello
context/3    hello! mein hon mai. aaj mein aap ki madad kai...
context/4                                                  Mai
context/5        Hello, aaj mai aapki madad kaise karsakti hun
Name: 827, dtype: object
response                                   mera naam mai hai .
context                                  aap ka naam kya hai ?
context/0                                           jee zarur!
context/1          kya mein aap se aik sawal pooch sakti hon ?
context/2      hi. aaj mein aap ki madad kaisay karsakti hon .
context/3                                                hello
context/4    hello! mein hon mai. aaj mein aap ki madad kai...
context/5                     

In [ ]:
# df2 = pd.read_csv(path+'/Reddit_clean.csv')

## Checking instance lengths to avoid memory errors during training
# count = 0
# for _, row in df2.iterrows():
#     if len(" ".join([row[i] for i in df2.columns])) > 1024:
#         count += 1
        
# print(count)
# df2.info()

In [ ]:
# new_df = pd.concat([df1, df2], ignore_index=True)
new_df = df1
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1321 entries, 0 to 1320
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   response   1321 non-null   object
 1   context    1321 non-null   object
 2   context/0  1321 non-null   object
 3   context/1  1321 non-null   object
 4   context/2  1321 non-null   object
 5   context/3  1321 non-null   object
 6   context/4  1321 non-null   object
 7   context/5  1321 non-null   object
dtypes: object(8)
memory usage: 82.7+ KB


In [ ]:
count = 0
df = new_df

# df = pd.DataFrame(columns=new_df.columns)

# for index, row in new_df.iterrows():
#     if len("".join([row[i] for i in new_df.columns])) > 1024:
#         count += 1
#         df.loc[len(df)] = row

# print(count)
# df.columns

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1321 entries, 0 to 1320
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   response   1321 non-null   object
 1   context    1321 non-null   object
 2   context/0  1321 non-null   object
 3   context/1  1321 non-null   object
 4   context/2  1321 non-null   object
 5   context/3  1321 non-null   object
 6   context/4  1321 non-null   object
 7   context/5  1321 non-null   object
dtypes: object(8)
memory usage: 82.7+ KB


In [ ]:
trn_df, val_df = train_test_split(df, test_size = 0.1)
trn_df.head()

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
40,ziyada tar larkiyan yeh nahi janteen ke un ke ...,mazeed sanitary masnooaat kya hai,aap ko is ke baray mein sharminda nahi hona ch...,Mai unhe khareedte huwe sharam mehsoos karti hun,Aap unhe qareebi store ya daraz ke zariye onli...,Mai ye sanitary ya mahwaari ke pads kahaan se ...,"apne haath sabun aur pani se dhowen aur, pad ...",jab mujhe mahwaari aati hai tou mujhe kya karn...
50,"mahvari ka pad, ya seedha pad, aik jaazib shai...",menstrual pads kya hote hai,"mahvari ka pad, ya seedha pad, aik jaazib shai...",sanitary napkins kya hote hai,mahvari ka cup mahvari se mutaliq hifzaan seha...,menstrual cups kya hote hai,tampon aik mahvari ki masnoaat hai jo haiz ke ...,tampons kya hai
339,hello,hello! aaj mein aap ki madad kaisay kar sakta ...,hello,kuch larkiyan inhen pehlay shuru karen gi. mah...,kya yeh sach hai ke larkiyon ko mahvari kuch d...,aap pehlay din se aakhri din tak naha sakte ha...,kya mahvari ke douran ghusal karna theek hai ?,aap mahvari ke douran apni mamool ki khoraak b...
174,hormonal adam tawazun/imbalance ka matlab hai ...,hormonal adam tawazun/imbalance aur fasid Mahv...,tanao ko kam karne ya un ka intizam karne aur ...,fasid mahvari ka ilaaj kaisay karen ?,aap ki mahvari be qaida hai agar yeh 21 din se...,mere mahvari be qaida kyun hain ?,aap ki mahvari be qaida hai agar yeh 21 din se...,mein kaisay jaan sakti hon ke agar mujhe be qa...
561,paidaiesh par qaboo panay ki golian mahvari ko...,aik mahvari aam tor par 3 se 8 din ke darmiyan...,aik ghair mard ke mahvari ko khatam honay mein...,aap pehlay din se aakhri din tak naha sakte ha...,kya aap haiz ke douran ghusal kar sakte hain ?,agar aap ke mahvari ko mutasir karne wali koi ...,kya hoga agar mera aik period cycle choot jaaye,mahvari ki be qaadgyon ki mojoodgi pcos ki ala...


In [ ]:
!nvidia-smi

Mon Apr  3 14:17:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.31       Driver Version: 465.31       CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA TITAN RTX    Off  | 00000000:01:00.0 Off |                  N/A |
| 41%   48C    P8    15W / 280W |   1392MiB / 24217MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
torch.cuda.empty_cache()

import gc
gc.collect()

30

# Constructing Model

In [ ]:
def construct_conv(row, tokenizer, eos = True):
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    conv = flatten(conv)
    return conv

class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):

        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

        directory = args.cache_dir
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            for _, row in df.iterrows():
                conv = construct_conv(row, tokenizer)
                self.examples.append(conv)

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

In [ ]:
# Cacheing and storing of data/checkpoints

def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
    return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []

    glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))

    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))

    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args.save_total_limit:
        return
    if args.save_total_limit <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args.save_total_limit:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
        shutil.rmtree(checkpoint)

In [ ]:
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    global Missed
    
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)


    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: # Error handling
                Missed += 1
                continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation of some model
def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    global Missed
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    Word_Error_Rate = WordErrorRate()
    ROGUE = ROUGEScore(rouge_keys=('rouge1', 'rouge2', 'rougeL'), accumulate='avg')

    # Custom BLEU
    custom_data_loader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=1, collate_fn=collate, drop_last = True
    )
    bleu_loss_2 = 0.0
    bleu_loss_4 = 0.0
    WER = 0.0
    rogue_loss_1 = {"precision":0.0, "fmeasure":0.0, "recall":0.0}
    rogue_loss_2 = {"precision":0.0, "fmeasure":0.0, "recall":0.0}
    rogue_loss_l = {"precision":0.0, "fmeasure":0.0, "recall":0.0}
    # METEOR = 0.0

    vals = 0
    for batch in tqdm(custom_data_loader, desc="BLEU"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)
        
        if inputs.shape[1] > 1024: # Error handling
            Missed += 1
            continue
        with torch.no_grad():
            predicted = model.generate(
                inputs, max_length=1000,
                pad_token_id=tokenizer.eos_token_id,  
                no_repeat_ngram_size=3,       
                do_sample=True, 
                top_k=100, 
                top_p=0.7,
                temperature = 0.8
                )
            
            y = tokenizer.decode(labels[:][0], skip_special_tokens=True).split()
            y_hat = tokenizer.decode(predicted[:][0], skip_special_tokens=True).split()

            # Word Error Rate
            WER_ = Word_Error_Rate(y_hat, y)
            WER += WER_

            # METEOR
            # METEOR_ = meteor_score([y_hat], y)
            # METEOR += METEOR_
            
            # Rogue
            temp = ROGUE(y_hat, y)
            rogue_loss_1["precision"] += temp['rouge1_precision']
            rogue_loss_1["fmeasure"] += temp['rouge1_fmeasure']
            rogue_loss_1["recall"] += temp['rouge1_recall']
            
            rogue_loss_2["precision"] += temp['rouge2_precision']
            rogue_loss_2["fmeasure"] += temp['rouge2_fmeasure']
            rogue_loss_2["recall"] += temp['rouge2_recall']
            
            rogue_loss_l["precision"] += temp['rougeL_precision']
            rogue_loss_l["fmeasure"] += temp['rougeL_fmeasure']
            rogue_loss_l["recall"] += temp['rougeL_recall']
#             rogue_loss_2 += temp['rogue2']
#             rogue_loss_l += temp['roguel']
#             print(temp)

            # BLEU 
            bleu_2 = sentence_bleu([y], y_hat, weights=(0.5, 0.5, 0, 0))
            bleu_4 = sentence_bleu([y], y_hat, weights=(0.25, 0.25, 0.25, 0.25))
            bleu_loss_2 += bleu_2
            bleu_loss_4 += bleu_4
        vals += 1

    # Perplexity Calculation
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)
        
        if inputs.shape[1] > 1024: # Error handling
            continue
        
        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"Perplexity": perplexity, 
              "Rogue-1-precision" : rogue_loss_1["precision"] / vals,
              "Rogue-1-fmeasure" : rogue_loss_1["fmeasure"] / vals,
              "Rogue-1-recall" : rogue_loss_1["recall"] / vals,
              "Rogue-2-precision" : rogue_loss_2["precision"] / vals,
              "Rogue-2-fmeasure" : rogue_loss_2["fmeasure"] / vals,
              "Rogue-2-recall" : rogue_loss_2["recall"] / vals,
              "Rogue-l-precision" : rogue_loss_l["precision"] / vals,
              "Rogue-l-fmeasure" : rogue_loss_l["fmeasure"] / vals,
              "Rogue-l-recall" : rogue_loss_l["recall"] / vals,
              "BLEU-2": bleu_loss_2 / vals, 
              "BLEU-4": bleu_loss_4 / vals, 
              "Word Error Rate:": float(WER) / vals}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

# Model Training

In [ ]:
# Main runner
def main(df_trn, df_val):
    args = Args()
    
    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir)
    model = AutoModelWithLMHead.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir=args.cache_dir,
    )
    model.to(args.device)
    
    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)

        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelWithLMHead.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelWithLMHead.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results

In [ ]:
main(trn_df, val_df)

04/03/2023 14:17:04 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
/home/sa05583/.local/lib/python3.8/site-packages/transformers/models/auto/modeling_auto.py:1132: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
04/03/2023 14:17:09 - INFO - __main__ -   Training/evaluation parameters <__main__.Args object at 0x7f1ecd03faf0>
04/03/2023 14:17:09 - INFO - __main__ -   Creating features from dataset file at cached
04/03/2023 14:17:10 - INFO - __main__ -   Saving features into cached file cached/gpt2_cached_lm_512
/home/sa05583/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future 

Epoch:   0%|          | 0/25 [00:00<?, ?it/s]

Iteration:   0%|          | 0/297 [00:00<?, ?it/s]

Iteration:   0%|          | 0/297 [00:00<?, ?it/s]

Iteration:   0%|          | 0/297 [00:00<?, ?it/s]

...

Input length of input_ids is 1017, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initiali

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

Evaluating:   0%|          | 0/33 [00:00<?, ?it/s]

04/03/2023 14:43:02 - INFO - __main__ -   ***** Eval results  *****
04/03/2023 14:43:02 - INFO - __main__ -     BLEU-2 = 0.9220827186714288
04/03/2023 14:43:02 - INFO - __main__ -     BLEU-4 = 0.9216217440336124
04/03/2023 14:43:02 - INFO - __main__ -     Perplexity = tensor(1.2226)
04/03/2023 14:43:02 - INFO - __main__ -     Rogue-1-fmeasure = tensor(0.9902)
04/03/2023 14:43:02 - INFO - __main__ -     Rogue-1-precision = tensor(0.9895)
04/03/2023 14:43:02 - INFO - __main__ -     Rogue-1-recall = tensor(0.9916)
04/03/2023 14:43:02 - INFO - __main__ -     Rogue-2-fmeasure = tensor(0.0448)
04/03/2023 14:43:02 - INFO - __main__ -     Rogue-2-precision = tensor(0.0448)
04/03/2023 14:43:02 - INFO - __main__ -     Rogue-2-recall = tensor(0.0448)
04/03/2023 14:43:02 - INFO - __main__ -     Rogue-l-fmeasure = tensor(0.9902)
04/03/2023 14:43:02 - INFO - __main__ -     Rogue-l-precision = tensor(0.9895)
04/03/2023 14:43:02 - INFO - __main__ -     Rogue-l-recall = tensor(0.9916)
04/03/2023 14:43:

{'Perplexity_': tensor(1.2226),
 'Rogue-1-precision_': tensor(0.9895),
 'Rogue-1-fmeasure_': tensor(0.9902),
 'Rogue-1-recall_': tensor(0.9916),
 'Rogue-2-precision_': tensor(0.0448),
 'Rogue-2-fmeasure_': tensor(0.0448),
 'Rogue-2-recall_': tensor(0.0448),
 'Rogue-l-precision_': tensor(0.9895),
 'Rogue-l-fmeasure_': tensor(0.9902),
 'Rogue-l-recall_': tensor(0.9916),
 'BLEU-2_': 0.9220827186714288,
 'BLEU-4_': 0.9216217440336124,
 'Word Error Rate:_': 0.006088755210908521}

In [ ]:
print(Missed)

14


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small', padding_side='left', use_multiprocessing = False)
model_mai = AutoModelWithLMHead.from_pretrained('output-small')

In [ ]:
# import time

# # Let's chat for 5 lines
# for step in range(5):
#     # encode the new user input, add the eos_token and return a tensor in Pytorch
#     new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
#     # print(new_user_input_ids)

#     # append the new user input tokens to the chat history
#     bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

#     start_time = time.time()
#     # generated a response while limiting the total chat history to 1000 tokens, 
#     chat_history_ids = model_mai.generate(
#         bot_input_ids, max_length=300,
#         pad_token_id=tokenizer.eos_token_id,  
#         no_repeat_ngram_size=3,       
#         do_sample=True, 
#         top_k=100, 
#         top_p=0.7,
#         temperature = 0.8
#     )
#     end_time = time.time()
    
#     print("Generation time:", end_time-start_time)
    
#     # pretty print last ouput tokens from bot
#     print("Mai: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

In [ ]:
if saveGeneral:
  pickle_out = open("mai_general_model.pkl", "wb")
else:
  pickle_out = open("mai_model.pkl", "wb")

pickle.dump(model_mai, pickle_out)
pickle_out.close()